## Setup & Imports

In [1]:
!pwd

/content


In [1]:
# copy custom scripts from MyDrive to use in Colab
# ref: https://stackoverflow.com/a/74791011
!cp -r /content/drive/MyDrive/24Su/PRIV_Capstone/scripts/. /content/

In [2]:
# if running notebook in Google Colab, %load will not work - https://github.com/googlecolab/colabtools/issues/42
# run the cell below this one to copy the contents of constants.py and have access to defined constants
%load constants.py

In [3]:
# data constants
DATA_DIR = "/content/drive/MyDrive/24Su/PRIV_Capstone/HARRISON"
DATA_LEN = 57383

# file path constants
INDICES_SPLIT_FILE_NAME = "data_indices_split.json"
IMG_PATHS_FILE_NAME = "data_list.txt"
GT_TAGS_FILE_NAME = "tag_list.txt"
EMBEDDINGS_FILE_NAME = "data_embeddings.txt"

# model constants
IMG_MODEL_NAME = "google/vit-base-patch16-224-in21k"

# df column constants
ID_COL = "img_id"
EMB_COL = "emb"
GT_COL = "gt_tags"
PRED_COL = "pred_tags"

# metric constants
ACCURACY = "acc"
PRECISION = "prec"
RECALL = "rec"

In [11]:
# !pip install -r requirements.txt

In [4]:
import os
import json
import numpy as np
import pandas as pd

In [5]:
from data_utils import load_image_tags
from eval_utils import evaluate_recommendations
from image_utils import get_image_from_fp, get_image_vectorizer_for_recommender
from index import create_index_from_df
from tag_recommender import TagRecommender

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.


## Prepare data

### Load in embeddings

In [6]:
emb_df = pd.read_json(os.path.join(DATA_DIR, EMBEDDINGS_FILE_NAME), lines=True)
emb_df.shape

(27410, 2)

### Split data

In [7]:
with open(os.path.join(DATA_DIR, INDICES_SPLIT_FILE_NAME)) as f:
    indices_split = json.load(f)

In [8]:
train_df = emb_df[emb_df[ID_COL].isin(indices_split["train"])]
val_df = emb_df[emb_df[ID_COL].isin(indices_split["val"])]

In [9]:
val_idx_mapping = {i:img_id for i,img_id in enumerate(val_df[ID_COL])}

## Create index

In [10]:
img_index, train_idx_mapping = create_index_from_df(train_df)

Creating index with vectors of shape: (17835, 768)


## Create recommender

In [11]:
tags_list = load_image_tags()

Loading image tags from file: /content/drive/MyDrive/24Su/PRIV_Capstone/HARRISON/tag_list.txt
Finished loading tags for 57383 images


In [12]:
recommender = TagRecommender(img_index, train_idx_mapping, tags_list)

### Pickle recommender

For easy sharing/loading

In [13]:
import pickle

In [15]:
with open(os.path.join(DATA_DIR, "hashtag_recommender.pkl"), "wb") as f:
    pickle.dump(recommender, f)

## Get recommendations

### By vectors

In [17]:
val_df.shape

(6861, 2)

In [18]:
recommender.get_tags_for_image_vector(np.array(val_df["emb"].head().to_list()))

[['sea', 'beach', 'friend', 'tbt', 'smile'],
 ['sun', 'sea', 'sunset', 'photography', 'summer'],
 ['sun', 'sea', 'home', 'wave', 'cloud'],
 ['instagram', 'weather', 'snow', 'life', 'friend'],
 ['sea', 'boat', 'ocean', 'passion', 'california']]

### By images

In [16]:
from data_utils import load_image_paths

In [17]:
img_fps = load_image_paths()

Loading image paths from file: /content/drive/MyDrive/24Su/PRIV_Capstone/HARRISON/data_list.txt
Finished loading 57383 image paths


In [18]:
val_img_fps = [img_fps[i] for i in indices_split["val"][:5]]
len(val_img_fps)

5

In [19]:
images = [get_image_from_fp(os.path.join(DATA_DIR, fp)) for fp in val_img_fps]
len(images)

5

In [20]:
img_vectorizer = get_image_vectorizer_for_recommender()

In [21]:
recommender.get_tags_for_image(images, img_vectorizer)

[['sea', 'beach', 'friend', 'tbt', 'smile'],
 ['sun', 'sea', 'sunset', 'photography', 'summer'],
 ['sun', 'sea', 'home', 'wave', 'cloud'],
 ['instagram', 'weather', 'snow', 'life', 'friend'],
 ['sea', 'boat', 'ocean', 'passion', 'california']]

#### Pickle image vectorizer

Recreate code from `image_utils.py` because we can't pickle local references to VIT_MODEL variable

In [23]:
from transformers import AutoImageProcessor, ViTModel

In [24]:
VIT_IMAGE_PROCESSOR = AutoImageProcessor.from_pretrained(IMG_MODEL_NAME)
VIT_MODEL = ViTModel.from_pretrained(IMG_MODEL_NAME)

Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.


In [28]:
def get_embeddings_from_model(images, image_processor, model):
    inputs = image_processor(images, return_tensors="pt")

    with torch.no_grad():
        outputs = model(**inputs)

    last_hidden_states = outputs.last_hidden_state
    embeddings = last_hidden_states[:, 0].cpu()
    return embeddings

def img_vectorizer(images):
    embeddings = get_embeddings_from_model(images, VIT_IMAGE_PROCESSOR, VIT_MODEL)
    return embeddings.numpy()

In [29]:
with open(os.path.join(DATA_DIR, "image_vectorizer.pkl"), "wb") as f:
    pickle.dump(img_vectorizer, f)

## Evaluate results

In [ ]:
pred_tags = recommender.get_tags_for_image_vector(np.array(val_df["emb"].to_list()))

In [ ]:
len(pred_tags)

6861

In [ ]:
pred_data = []
for i,pred_tag_list in enumerate(pred_tags):
    img_id = val_idx_mapping[i]
    pred_data.append({ID_COL: img_id, "gt_tags": tags_list[img_id], "pred_tags": pred_tag_list})
len(pred_data)

6861

In [ ]:
pred_df = pd.DataFrame(pred_data)
pred_df.head()

,img_id,gt_tags,pred_tags
0,0,"[sea, instapic, instagram, trip, travel]","[sea, beach, friend, tbt, smile]"
1,1,[sea],"[sun, sea, sunset, photography, summer]"
2,6,"[church, road, nikon, photograph, sea, beach]","[sun, sea, home, wave, cloud]"
3,13,"[tagsforlikes, dance, sweet, sea, photooftheday]","[instagram, weather, snow, life, friend]"
4,18,"[sea, travel, photo]","[sea, boat, ocean, passion, california]"


In [ ]:
eval_results = evaluate_recommendations(pred_df)

In [ ]:
with open(os.path.join(DATA_DIR, "eval_results_27499.json"), "w") as f:
    json.dump(eval_results, f, indent=2)

In [ ]:
eval_results

{'acc': {1: 0.39411164553272116,
  2: 0.48826701646990234,
  3: 0.5356362046348929,
  4: 0.5653694796676869,
  5: 0.5878151872904824},
 'prec': {1: 0.39411164553272116,
  2: 0.2979157557207404,
  3: 0.2426517028615848,
  4: 0.20781713064179175,
  5: 0.18248554632463684},
 'rec': {1: 0.1294303782825864,
  2: 0.18327965982098032,
  3: 0.21527335318327886,
  4: 0.23860762348956488,
  5: 0.2555624382581094}}